In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import *
from keras.models import load_model


import os
import time
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

#variables para guardar los resultados
tipo = pruebas[4]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Dataset de Amazon

In [5]:
print tipo
print dataset_name
print data_path
print dims

sda
amazon
data
2000


In [7]:
# cargando dataset de Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [8]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


In [9]:
# se divide el dataset para los datos de entrenamiento y validacion del SDA
X_train, X_val, _, _ = train_test_split(X, np.zeros(X.shape[0]), test_size=0.2, random_state=42)

In [10]:
sda_paths = os.path.join(models_path, tipo, "%s_paths.pkl" % dataset_name)

if os.path.exists(sda_paths):
    print "Cargando rutas de modelos adaptados."
    saved_paths = joblib.load(sda_paths)
else:
    print "Creando modelos de adaptacion..."

    noises = [0.5, 0.8]
    layers = [[int(dims/2)], [int(dims/2), int(dims/4)], [int(dims/2), int(dims/4), int(dims/5)]]
    epochs = [50, 100, 250]

    parametros = {
        'noise': noises,
        'layers': layers,
        'epochs': epochs,
    }
    
    saved_paths = sda_pseudo_grid_search(X_train, X_val, parametros, models_path, tipo, dataset_name)

    print "Modelos creados.\n"
    
    joblib.dump(saved_paths, sda_paths)
    print "Guardando rutas en %s" % sda_paths

Creando modelos de adaptacion...
pr: 0.500 - epochs: 50 - layers: [1000]
	Entrenando autoencoder...


KeyboardInterrupt: 

In [ ]:
saved_paths

In [ ]:
best_sda_paths = os.path.join(models_path, tipo, "best_%s_paths.pkl" % dataset_name )
if os.path.exists(best_sda_paths):
    #se cargan las rutas de los modelos guardados
    domain_sda_paths = joblib.load(best_sda_paths)   
else:
    domain_sda_paths = []
    #por cada dominio
    for domain in domains:
        #por cada modelo
        print "Obteniendo mejor clasificador para %s..." % domain
        
        i = 1
        best_score = 0
        best_encoder = None
        
        for temp_path in saved_paths:
            #autoencoder = load_model(temp_path['autoencoder'])
            encoder = load_model(temp_path['encoder'])

        
            X_tr = np.asarray(labeled[domain]['X_tr'].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[domain]['X_ts'].todense())
            
            #se adapta el dominio segun el modelo
            X_deep = encoder.predict(X_tr)

            #se crea un clasificador y se obtiene su score
            new_clf = get_best_score(X_deep, y_tr, clasifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "%d - score: %.4f" % (i, new_score)
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_encoder = encoder
                
            i= i+1
            
        best_encoder_path = os.path.join(models_path, tipo, "%s_%s_encoder.pkl" % (dataset_name, domain))
        print "Guardando en %s" % best_encoder_path
        encoder.save(best_encoder_path)
        
        domain_sda_paths.append(best_encoder_path)
        
    print "Guardando rutas en %s" % best_sda_paths
    joblib.dump(domain_sda_paths, best_sda_paths)
    
print "\nOperacion terminada.\n"

In [ ]:
domain_sda_paths

In [ ]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_sda_path = os.path.join(models_path, tipo, "%s_%s_encoder.pkl" % (dataset_name, domain))
    
    if os.path.exists(best_sda_path):
        print "Cargando mejor modelo para %s" % domain
        encoder = load_model(best_sda_path)
        
        print "Adaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        X_ts = np.asarray(labeled[domain]['X_ts'].todense())
        
        
        tr_reps = encoder.predict(X_tr)
        ts_reps = encoder.predict(X_ts)

        adapted[domain] = {
            'X_tr': tr_reps,
            'X_ts': ts_reps
        }
        
        i = i+1
    else:
        print "Generar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

In [71]:
df = pd.DataFrame(columns=dataframe_columns)

i = 0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr, n_jobs=4)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "Pruebas completadas."

Tarea 1 de 12
0.28  - 0.368
Tarea 2 de 12
0.2875  - 0.360
Tarea 3 de 12
0.3225  - 0.420
Tarea 4 de 12
0.24  - 0.397
Tarea 5 de 12
0.2875  - 0.410
Tarea 6 de 12
0.3225  - 0.360
Tarea 7 de 12
0.24  - 0.380
Tarea 8 de 12
0.28  - 0.365
Tarea 9 de 12
0.3225  - 0.432
Tarea 10 de 12
0.24  - 0.377
Tarea 11 de 12
0.28  - 0.310
Tarea 12 de 12
0.2875  - 0.385
Pruebas completadas.


In [72]:
df

,Adaptacion,Dataset,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,Baseline,Amazon,k->d,kitchen,dvd,28.00,36.75,8.75
1,Baseline,Amazon,k->e,kitchen,electronics,28.75,36.00,7.25
2,Baseline,Amazon,k->b,kitchen,books,32.25,42.00,9.75
3,Baseline,Amazon,d->k,dvd,kitchen,24.00,39.75,15.75
4,Baseline,Amazon,d->e,dvd,electronics,28.75,41.00,12.25
5,Baseline,Amazon,d->b,dvd,books,32.25,36.00,3.75
6,Baseline,Amazon,e->k,electronics,kitchen,24.00,38.00,14.00
7,Baseline,Amazon,e->d,electronics,dvd,28.00,36.50,8.50
8,Baseline,Amazon,e->b,electronics,books,32.25,43.25,11.00
9,Baseline,Amazon,b->k,books,kitchen,24.00,37.75,13.75


In [76]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."